### Visualizar en mapa las diferentes tiendas que tienen productos en especifico y los locales en renta

In [173]:
import ETL as etl  
import pandas as pd

In [174]:
nombre_archivo = './datasets/09-2023_02.csv'
nombre_productos = ['JITOMATE','AGUACATE','LECHUGA','PAPA']
presentaciones = ['1 KG. SALADETTE/ HUAJE O TOMATE SALADETTE/ HUAJE','1 KG. HASS','PZA. ROMANA','1 KG. ALFA/BLANCA']
fecha = '2023-09-28'

In [175]:
df = etl.ETL_ARCHIVO(nombre_archivo,nombre_productos,presentaciones,fecha)
df2 = pd.read_csv('./datasets/inmuebles24Corregido.csv', sep=',' , encoding='utf-8')

In [176]:
df.head(5)

,NOMBRECOMERCIAL,PRECIO,FECHAREGISTRO,PRESENTACIÓN,MARCA,LATITUD,LONGITUD,PRODUCTO
77176,BODEGA AURRERA EXPRESS SUCURSAL AZCAPOTZALCO,32.9,2023-09-28,1 KG. SALADETTE/ HUAJE O TOMATE SALADETTE/ HUAJE,S/M,19.482528,-99.184364,JITOMATE
77662,MERCADO SORIANA SUCURSAL AZCAPOTZALCO,30.9,2023-09-28,1 KG. SALADETTE/ HUAJE O TOMATE SALADETTE/ HUAJE,S/M,19.478250,-99.187038,JITOMATE
78592,WALMART SUCURSAL CUITLAHUAC,32.9,2023-09-28,1 KG. SALADETTE/ HUAJE O TOMATE SALADETTE/ HUAJE,S/M,19.471594,-99.164840,JITOMATE
110976,BODEGA AURRERA SUCURSAL CANTIL,27.9,2023-09-28,1 KG. SALADETTE/ HUAJE O TOMATE SALADETTE/ HUAJE,S/M,19.330481,-99.154328,JITOMATE
111872,LACOMER SUCURSAL MIGUEL ANGEL DE QUEVEDO,38.9,2023-09-28,1 KG. SALADETTE/ HUAJE O TOMATE SALADETTE/ HUAJE,S/M,19.345444,-99.172181,JITOMATE


In [177]:
df2.head(5)

,precio,ubicacion,tamanio,latitud,longitud
0,8000,"BOLAÑOS, Felipe Pescador, Cuauhtémoc",30,19.454,-99.12
1,16000,"Huitzilopochtli Lte 14 Mz 19, Adolfo Ruiz Co...",35,19.320,-99.15
2,9000,"Lucerna 65, Juárez, Cuauhtémoc",8,19.430,-99.15
3,18000,"Montecito 38, Nápoles, Benito Juárez",40,19.394,-99.17
4,7500,"Petén, Narvarte Oriente, Benito Juárez",12,19.387,-99.15


In [178]:
df = df.groupby(['PRODUCTO','NOMBRECOMERCIAL','LATITUD','LONGITUD']).agg({'PRECIO':'mean'}).reset_index()

In [179]:
# Contar la cantidad de productos distintos que tiene cada tienda
df['COUNT'] = df.groupby('NOMBRECOMERCIAL')['PRODUCTO'].transform('nunique')

# Filtrar las tiendas que tienen todos los productos deseados
tiendas_con_todos_los_productos = df[(df['COUNT'] == len(nombre_productos))]

# Eliminar la columna de conteo
tiendas_con_todos_los_productos = tiendas_con_todos_los_productos.drop(columns=['COUNT'])

# Agrupar por tienda y calcular la suma de precios
total_precios_por_tienda = tiendas_con_todos_los_productos.groupby('NOMBRECOMERCIAL')['PRECIO'].sum()

# Mantener las coordenadas de las tiendas
coordenadas_tienda = tiendas_con_todos_los_productos.groupby('NOMBRECOMERCIAL')[['LATITUD', 'LONGITUD']].first()

# Combinar los resultados en un nuevo DataFrame
resultado_final = pd.concat([total_precios_por_tienda, coordenadas_tienda], axis=1)

# Ordenar las tiendas por precio total de menor a mayor
resultado_final = resultado_final.sort_values(by='PRECIO')



In [180]:
# Reiniciar el índice del DataFrame
resultado_final.reset_index(inplace=True)

# Ordenar las tiendas por precio total de menor a mayor
resultado_final = resultado_final.sort_values(by='PRECIO')

# Mostrar los resultados
(resultado_final)

,NOMBRECOMERCIAL,PRECIO,LATITUD,LONGITUD
0,CHEDRAUI SUCURSAL ANFORA,122.40,19.437353,-99.110088
1,U.N.A.M. SUCURSAL 02 CD UNIVERSITARIA,127.73,19.322043,-99.176224
2,MEGA SORIANA SUCURSAL ASTURIAS,137.20,19.407812,-99.132654
3,SORIANA HIPER SUCURSAL CONSULADO (418),141.60,19.449572,-99.100376
4,BODEGA AURRERA SUCURSAL LA VIGA,143.70,19.414749,-99.127874
5,BODEGA AURRERA SUCURSAL CANTIL,147.70,19.330481,-99.154328
6,BODEGA AURRERA EXPRESS SUCURSAL LOS GALLOS,149.70,19.455788,-99.162735
7,BODEGA AURRERA EXPRESS SUCURSAL AZCAPOTZALCO,153.70,19.482528,-99.184364
8,WALMART SUCURSAL PORTAL CENTRO,153.70,19.419472,-99.130269
9,WALMART SUCURSAL BALBUENA,154.70,19.418067,-99.105639


In [188]:
import folium
from ipywidgets import interact
from IPython.display import display

# Crear una función para generar el mapa
def crear_mapa(precio_min, precio_max, tamanio_min, tamanio_max):
    # Crear un mapa centrado en una ubicación inicial
    m = folium.Map(location=[19.4, -99.15], zoom_start=12)

    # Iterar a través de los datos del primer conjunto
    for i, row in enumerate(df2).iterrows:
        precio = row['precio']
        latitud = row['latitud']
        longitud = row['longitud']
        if precio_min <= precio <= precio_max:
            color = 'green'
        elif precio_max < precio <= 150:
            color = 'yellow'
        else:
            color = 'red'
        icono = folium.Icon(color=color, icon='shopping-cart', prefix='fa')
        folium.Marker([latitud, longitud], icon=icono).add_to(m)

    # Iterar a través de los datos del segundo conjunto
    for i, row in (resultado_final).iterrows:
        tamanio = row['tamanio']
        latitud = row['latitud']
        longitud = row['longitud']
        if tamanio_min <= tamanio <= tamanio_max:
            color = 'green'
        elif tamanio_max < tamanio <= 8000:
            color = 'orange'
        else:
            color = 'red'
        icono = folium.Icon(color=color, icon='home', prefix='fa')
        folium.Marker([latitud, longitud], icon=icono).add_to(m)

    # Mostrar el mapa
    display(m)

# Crear controles deslizantes interactivos para ajustar los filtros
interact(crear_mapa, precio_min=(0, 200, 10), precio_max=(0, 200, 10),
         tamanio_min=(0, 100, 5), tamanio_max=(0, 100, 5))


interactive(children=(IntSlider(value=100, description='precio_min', max=200, step=10), IntSlider(value=100, d…

<function __main__.crear_mapa(precio_min, precio_max, tamanio_min, tamanio_max)>